<h1><center>Ingénierie Mathématique 3, Devoir Maison</center></h1>
<h1><center>Janvier 2025</center></h1>

<h3><center>Total: 15 points<br>
    <br>
    Début: 1 Janvier 2025<br>
    <br>
    Fin: 24 Janvier 2025</center></h3>
    
   
<h3><font color='red'>Consignes:</font></h3>

<font color='red'>Aucun devoir soumis après le 20 janvier ne sera pris en compte. Les devoirs qui ne respectent pas strictement les consignes reprises sur le site web (http://www.augustincosse.com/wp-content/uploads/2023/03/consignesSoumission2.pdf) ne seront pas pris en compte. En particulier, la soumission consistera en un __unique__ fichier ".ipynb" envoye par email dans un email dont l'objet sera "INFO2 Ingenierie Math DM"
</font>

## Question 1 (5pts) Un premier modèle

On considère le jeu de données MNIST ci-dessous (les images peuvent être téléchargées via scikit learn ou via keras). On souhaite commencer par entraîner un modèle de régression logistique permettant de différencier les 1 des 0. Pour ce faire on procédera comme suit:

- Extraire à partir des données ci-dessous, les images représentant des $1$ ou des $0$.
- Séparer les données en un ensemble de test et un ensemble d'entraînement (on gardera 10% des données pour l'ensemble test)
- Compléter la fonction "binary_cross_entropy" afin que celle-ci retourne la valeur de l'entropie binaire croisée ainsi que le gradient de cette fonction en une image donnée et pour un vecteur de coefficients de régression $\mathbf{w}$ donne.
- Compléter ensuite la fonction “optimisation" afin qu'elle implémente une descente de gradient sur la fonction d'entropie binaire croisée. On souhaite  renvoyer en sortie le vecteur des coefficients de régression ainsi que (1) le taux de classification (en pourcentage de données correctement classées sur le nombre de données totales) sur les ensembles d'entraînement et de test.  



In [1]:
from sklearn.datasets import fetch_openml
import numpy as np

# Charger les données MNIST
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

# Normaliser les données et convertir les labels
X = X / 255.0
mask = (y == '0') | (y == '1')
X, y = X[mask], y[mask].astype(int)

# Fonction sigmoïde
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Fonction binaire croisée et son gradient
def binary_cross_entropy(X, y, w):
    '''
    Calcule la valeur de l'entropie binaire croisée et son gradient.
    l(w) = - sum_i t_i*log(sigma(y(x_i))) + (1-t_i)*log(1-sigma(y(x_i)))
    ainsi que le gradient de cette fonction.
    '''
    m = X.shape[0]
    z = np.dot(X, w)
    predictions = sigmoid(z)

    # Calcul de l'entropie binaire croisée
    bce_fun = -np.mean(y * np.log(predictions + 1e-8) + (1 - y) * np.log(1 - predictions + 1e-8))

    # Calcul du gradient
    bce_grad = np.dot(X.T, (predictions - y)) / m

    return bce_fun, bce_grad

# Fonction d'optimisation
def optimisation(w_init, eta, X_train, y_train, X_test, y_test, epochs=100):
    '''
    Effectue une descente de gradient pour minimiser l'entropie binaire croisée.
    Retourne :
      - Le vecteur des coefficients optimaux w_opt
      - Les taux de classification sur les ensembles d'entraînement et de test
    '''
    w = w_init

    for epoch in range(epochs):
        # Calcul de la perte et du gradient
        loss, grad = binary_cross_entropy(X_train, y_train, w)

        # Mise à jour des coefficients
        w -= eta * grad

        # Affichage de la perte toutes les 10 itérations
        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Loss = {loss:.4f}")

    # Prédiction
    def predict(X, w):
        return (sigmoid(np.dot(X, w)) >= 0.5).astype(int)

    y_train_pred = predict(X_train, w)
    y_test_pred = predict(X_test, w)

    # Taux de classification
    rate_bce_training = np.mean(y_train_pred == y_train) * 100
    rate_bce_test = np.mean(y_test_pred == y_test) * 100

    return w, rate_bce_training, rate_bce_test

# Initialisation et test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
w_init = np.zeros(X_train.shape[1])
eta = 0.1

w_opt, rate_bce_training, rate_bce_test = optimisation(w_init, eta, X_train, y_train, X_test, y_test)

print(f"Taux de classification sur l'ensemble d'entraînement : {rate_bce_training:.2f}%")
print(f"Taux de classification sur l'ensemble de test : {rate_bce_test:.2f}%")


Epoch 0: Loss = 0.6931
Epoch 10: Loss = 0.1124
Epoch 20: Loss = 0.0673
Epoch 30: Loss = 0.0501
Epoch 40: Loss = 0.0408
Epoch 50: Loss = 0.0349
Epoch 60: Loss = 0.0308
Epoch 70: Loss = 0.0278
Epoch 80: Loss = 0.0255
Epoch 90: Loss = 0.0236
Taux de classification sur l'ensemble d'entraînement : 99.76%
Taux de classification sur l'ensemble de test : 99.73%


## Question 2 (5pts) Un petit réseau convolutif

On souhaite à présent ameliorer le resultat de la question 1 a l'aide d'un réseau convolutionnel. Pour ce faire, on souhaite utiliser la librairie Keras (voir https://keras.io/) et en particulier, le modele de reseau convolutionnel (https://keras.io/api/layers/convolution_layers/). Un réseau convolutionnel fonctionne en "filtrant" les images à l'aide de différents filtres dont les coefficients sont appris lors de l'étape d'entraînement. Un réseau convolutionnel efficace est typiquement constitué d'une succession de couches convolutives et de pooling (voir par exemple https://www.tensorflow.org/tutorials/images/cnn).

Chaque couche convolutive déplace un filtre (dont les coefficients sont fixés lors de l'étape d'entraînement) sur les sorties des couches précédentes


<img src="same_padding_no_strides.gif" alt="stackoverflow.com" width=304 height=142>


Afin de réduire la dimension des sorties des couches successives, on alterne généralement entre des couches convolutives et des couches de pooling (équivalentes à un sous-échantillonnage) qui retiennent pour une région donnée, uniquement les pixels de plus forte intensité (afin de conserver une trace du contraste). Ces couches sont de la forme suivante:


<img src="maxPool.png" alt="stackoverflow.com" width=504 height=142>


En utilisant le modele Sequential() de keras et la fonction add() permettant d'ajouter des couches convolutionnelles ou de pooling, ainsi que la fonction compile() permettant d'entrainer le reseau, et en vous aidant de la documentation de tensorFlow (https://www.tensorflow.org/tutorials/images/cnn), entrainer un (petit) reseau convolutionnel sur les images 0 et 1 des donnees MNIST de la question 1 (A nouveau, on retiendra environ 10% de donnees pour l'ensemble test).

In [2]:
from sklearn.datasets import fetch_openml
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# Charger les données MNIST
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

# Normaliser les données et convertir les labels
X = X / 255.0
mask = (y == '0') | (y == '1')
X, y = X[mask], y[mask].astype(int)

# Reshape des données pour les couches convolutives
X = X.reshape(-1, 28, 28, 1)
y = to_categorical(y, num_classes=2)

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Création du modèle convolutionnel
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entraînement du modèle
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Évaluation du modèle
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f"Taux de classification sur l'ensemble d'entraînement : {train_accuracy * 100:.2f}%")
print(f"Taux de classification sur l'ensemble de test : {test_accuracy * 100:.2f}%")


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


416/416 ━━━━━━━━━━━━━━━━━━━━ 14s 30ms/step - accuracy: 0.9820 - loss: 0.0485 - val_accuracy: 0.9986 - val_loss: 0.0037
Epoch 2/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 14s 33ms/step - accuracy: 0.9991 - loss: 0.0020 - val_accuracy: 0.9993 - val_loss: 0.0019
Epoch 3/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 19s 29ms/step - accuracy: 0.9996 - loss: 9.3642e-04 - val_accuracy: 1.0000 - val_loss: 5.2040e-04
Epoch 4/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 12s 29ms/step - accuracy: 0.9997 - loss: 6.9381e-04 - val_accuracy: 0.9986 - val_loss: 0.0017
Epoch 5/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 12s 29ms/step - accuracy: 0.9995 - loss: 0.0011 - val_accuracy: 0.9993 - val_loss: 0.0071
Epoch 6/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 14s 33ms/step - accuracy: 0.9995 - loss: 0.0010 - val_accuracy: 0.9993 - val_loss: 0.0050
Epoch 7/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 0.9997 - loss: 4.4029e-04 - val_accuracy: 1.0000 - val_loss: 3.8245e-04
Epoch 8/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 21s 29ms/step - accuracy: 0.9998 - loss: 

## Question 3 (5pts) Astuce du Noyau

On souhaite à présent entraîner un modèle de régression linéaire dans l'espace noyau sur les images de 0 et de 1. Afin d'atteindre cet objectif, on procédera comme suit:

- Sélectionner un sous-ensemble d'équilibre d'images de 0 et de 1 (on commencera par exemple par prendre 50 images du chiffre 0 et 50 images du chiffre 1).
- Compléter la fonction G(x, y) qui renvoie la valeur d'un noyau Gaussien de paramètre sigma, aux points $x$ et $y$.
- Compléter la fonction "optimize" a l'aide d'une descente de gradient dans l'espace noyau afin qu'elle retourne le vecteur des coefficients $\lambda_j$ d'un modele du type
$$y(x) = \sum_{j=1}^N \lambda_j G(x, x_j)$$

- Comme pour la question 2, on souhaite implémenter la fonction "optimize" de façon à ce qu'elle retourne la liste des taux de classification correcte (sur l' ensemble d'entraînement et de test) pour chacune des itérations.


In [3]:
from sklearn.datasets import fetch_openml
import numpy as np
from sklearn.model_selection import train_test_split

def G(x, y, sigma):
    '''
    La fonction doit renvoyer la valeur du noyau gaussien de
    variance sigma aux points x et y
    '''
    diff = x - y
    return np.exp(-np.dot(diff, diff) / (2 * sigma**2))

def optimize(X, y, sigma, lr=0.01, epochs=100):
    '''
    La fonction doit renvoyer le vecteur lambda d'un modele de type
    y(x) = sum_i lambda_i G(x, x_i, sigma) ou G est le noyeau Gaussien base sur
    l'astuce du noyeau
    '''
    N = X.shape[0]
    lbda = np.zeros(N)

    def kernel_matrix(X1, X2, sigma):
        """Calcule la matrice de noyau Gaussien entre deux ensembles."""
        K = np.zeros((X1.shape[0], X2.shape[0]))
        for i in range(X1.shape[0]):
            for j in range(X2.shape[0]):
                K[i, j] = G(X1[i], X2[j], sigma)
        return K

    K_train = kernel_matrix(X, X, sigma)

    for epoch in range(epochs):
        predictions = np.dot(K_train, lbda)
        errors = predictions - y

        # Descente de gradient
        grad = np.dot(K_train.T, errors) / N
        lbda -= lr * grad

        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Gradient norm = {np.linalg.norm(grad):.4f}")

    # Taux de classification
    def predict(X_train, X_test, lbda, sigma):
        K_test = kernel_matrix(X_test, X_train, sigma)
        return np.sign(np.dot(K_test, lbda))

    y_train_pred = predict(X, X, lbda, sigma)
    y_test_pred = predict(X, X_test, lbda, sigma)

    rate_training = np.mean(y_train_pred == y) * 100
    rate_test = np.mean(y_test_pred == y_test) * 100

    return lbda, rate_training, rate_test

# Charger les données MNIST
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

# Normaliser les données et convertir les labels
X = X / 255.0
mask = (y == '0') | (y == '1')
X, y = X[mask], y[mask].astype(int)

# Sélection d'un sous-ensemble équilibré
def select_balanced_subset(X, y, n_samples=50):
    idx_0 = np.where(y == 0)[0][:n_samples]
    idx_1 = np.where(y == 1)[0][:n_samples]
    idx = np.hstack([idx_0, idx_1])
    np.random.shuffle(idx)
    return X[idx], y[idx]

X_subset, y_subset = select_balanced_subset(X, y)

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.2, random_state=42)

# Entraînement du modèle noyau gaussien
sigma = 1.0
lbda, rate_training, rate_test = optimize(X_train, y_train, sigma)

print(f"Taux de classification sur l'ensemble d'entraînement : {rate_training:.2f}%")
print(f"Taux de classification sur l'ensemble de test : {rate_test:.2f}%")


Epoch 0: Gradient norm = 0.0846
Epoch 10: Gradient norm = 0.0845
Epoch 20: Gradient norm = 0.0844
Epoch 30: Gradient norm = 0.0843
Epoch 40: Gradient norm = 0.0841
Epoch 50: Gradient norm = 0.0840
Epoch 60: Gradient norm = 0.0839
Epoch 70: Gradient norm = 0.0838
Epoch 80: Gradient norm = 0.0837
Epoch 90: Gradient norm = 0.0836
Taux de classification sur l'ensemble d'entraînement : 52.50%
Taux de classification sur l'ensemble de test : 40.00%
